In [ ]:
!pip install rasa==1.10.5

     |████████████████████████████████| 511 kB 5.1 MB/s 
     |████████████████████████████████| 661 kB 43.0 MB/s 
     |████████████████████████████████| 104 kB 50.4 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 509 kB 45.3 MB/s 
     |████████████████████████████████| 3.2 MB 43.4 MB/s 
     |████████████████████████████████| 448 kB 54.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.6 MB/s 
     |████████████████████████████████| 47 kB 4.8 MB/s 
     |████████████████████████████████| 266 kB 53.9 MB/s 
     |████████████████████████████████| 5.1 MB 31.5 MB/s 
     |████████████████████████████████| 40 kB 5.9 MB/s 
     |████████████████████████████████| 990 kB 44.2 MB/s 
     |████████████████████████████████| 3.0 MB 40.1 MB/s 
     |████████████████████████████████| 60 kB 7.4 MB/s 
     |████████████████████████████████| 174 kB 54.8 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |█████████████████

In [ ]:
!python -m spacy download en

In [ ]:
!pip install nest_asyncio==1.3.3

In [ ]:
!pip install python-restcountries
from restcountries import RestCountryApiV2 as rapi

In [ ]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()

print("event loop ready")

event loop ready


In [ ]:
from rasa.cli.scaffold import create_initial_project

In [ ]:
project = 'chatbot_project'
create_initial_project(project)

In [ ]:
os.chdir(project)
print(os.listdir("."))

['__init__.py', 'tests', 'actions.py', 'endpoints.yml', 'config.yml', '__pycache__', 'domain.yml', 'credentials.yml', 'data']


In [ ]:
config = "config.yml"
training_files = "data/"
domain = 'domain.yml'
output = 'models/'

print(config, training_files, domain, output)

config.yml data/ domain.yml models/


In [ ]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

In [ ]:
from rasa.jupyter import chat

endpoints = 'endpoints.yml'

chat(model_path, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
hey
Hey! How are you?
great
Great, carry on!
/stop


In [ ]:
%%writefile data/nlu.md

## intent:check_capital
- what is the capital of [USA](country1)?
- can you tell me the capital of [Russia](country1)?
- Please tell me the capital of [Greece](country1)
- do you know what is the capital of [Sweden](country1)?
- i want to know the capital of [Australia](countr1)
- capital of [Japan](country1)

##intent: check_population
- what is the population of [USA](country2)?
- can you tell me the population of [Russia](country2)?
- Please tell me the population of [Greece](country2)
- do you know what is the population of [Sweden](country2)?
- i want to know the population of [Australia](country2)
- population of [Japan](country2)


## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?


Overwriting data/nlu.md


In [ ]:
%%writefile domain.yml

intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge
  - check_capital
  - check_population

slots:
  country1:
    type: text
    mappings:
    - type: from_entity
      entity: country1
  country2:
    type: text
    mappings:
    - type: from_entity
      entity: country2
  countries: 
    type: list
    influence_conversation: false

forms:
  capital_form:
    required_slots:
        - country1
  population_form:
    required_slots:
        - country2     
        
actions:
  - utter_greet
  - utter_goodbye
  - action_get_countries
  - action_know_capital
  - action_know_population
  - action_all_slot_reset
  - validate_capital_form
  - validate_population_form

    
responses:
  utter_greet:
  - text: "Hey! How may I help you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great, carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_iamabot:
  - text: "I am a bot, powered by Rasa."

  utter_select_country:
  - text: "Please select a country.\n {countries}"

entities:
  - country1
  - country2

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true


Overwriting domain.yml


In [ ]:
%%writefile data/stories.yml

stories:

- story: greet 
  steps:
  - intent: greet
  - action: utter_greet

- story: user goodbye
  steps:
  - intent: goodbye
  - action: utter_goodbye

- story: check capital
  steps:
  - intent: check_capital
  - action: action_get_countries
  - action: capital_form
  - active_loop: capital_form
  - active_loop: null
  - action: action_know_capital
  - action: action_all_slot_reset

- story: check population
  steps:
  - intent: check_population
  - action: action_get_countries
  - action: population_form
  - active_loop: population_form
  - active_loop: null
  - action: action_know_population
  - action: action_all_slot_reset

In [ ]:
%%writefile actions.py
# This files contains your custom actions which can be used to run
# custom Python code.
#
# See this guide on how to implement these action:
# https://rasa.com/docs/rasa/core/actions/#custom-actions/


# This is a simple example for a custom action which utters "Hello World!"

from rasa_sdk.events import AllSlotsReset,SlotSet
from rasa_sdk import Action, Tracker,events,FormValidationAction
from typing import Dict, Text, Any, List, Union, Optional
from rasa_sdk.types import DomainDict
from rasa_sdk.executor import CollectingDispatcher
from restcountries import RestCountryApiV2 as rapi
import requests
import json

class ActionAllSlotReset(Action):

     def name(self) -> Text:
            return "action_all_slot_reset"

     def run(self, dispatcher: CollectingDispatcher,
             tracker: Tracker,
             domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

         return [AllSlotsReset()]

class ValidateCapitalForm(FormValidationAction):
    def name(self) -> Text:
        return "validate_capital_form"

    def validate_countryname(self, value: Text, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> Optional[Text]:
    	country = tracker.get_slot('country1')
      API_address = 'https://qcooc59re3.execute-api.us-east-1.amazonaws.com/dev/getCountries'
			# get countries using API
			try:
				json_data = requests.get(API_address).json()
			except requests.exceptions.HTTPError as Exception:
				print (Exception)    
			else: countries =  json_data['body']
    	if (country in countries):
    		return {"country1" : value}
    	else :
    		dispatcher.utter_message(text="Please select another country")
    		return{"country1" : None}


 def validate_countryname(self, value: Text, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> Optional[Text]:
    	country = tracker.get_slot('country2')
      API_address = 'https://qcooc59re3.execute-api.us-east-1.amazonaws.com/dev/getCountries'
			# get countries using API
			try:
				json_data = requests.get(API_address).json()
			except requests.exceptions.HTTPError as Exception:
				print (Exception)    
			else: countries =  json_data['body']
    	if (country in countries):
    		return {"country2" : value}
    	else :
    		dispatcher.utter_message(text="Please select another country")
    		return{"country2" : None}

class ActionGetCountries(Action):

     def name(self) -> Text:
            return "action_get_countries"

     def run(self, dispatcher: CollectingDispatcher,
             tracker: Tracker,
             domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
     	api_address = 'https://qcooc59re3.execute-api.us-east-1.amazonaws.com/dev/getCountries'
       try:
				json_data = requests.get(API_address).json()
			except requests.exceptions.HTTPError as Exception:
				print (Exception)    
			else: countries =  json_data['body']
     	
     	return [SlotSet('countries',countries)]


class ActionGetCapital(Action):

     def name(self) -> Text:
            return "action_know_capital"

     def run(self, dispatcher: CollectingDispatcher,tracker: Tracker,domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
       API_address = "https://qcooc59re3.execute-api.us-east-1.amazonaws.com/dev/getCapital"
       country = tracker.get_slot('country1')
			 data = {"country": country}
			 headers = {"content-Type":"application/json"}
			 try:
			  	json_data = requests.post(API_address, headers=headers, json=data).json()
			 except requests.exceptions.HTTPError as Exception:
				 print (Exception) 
			 else: capital =  json_data['body']['capital']
       dispatcher.utter_message(text = capital+"is the capital of "+country)
       return []

class ActionGetPopulation(Action):

     def name(self) -> Text:
            return "action_know_population"

     def run(self, dispatcher: CollectingDispatcher,tracker: Tracker,domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
       API_address = "https://qcooc59re3.execute-api.us-east-1.amazonaws.com/dev/getPopulation"
       country = tracker.get_slot('country2')
			 data = {"country": country}
			 headers = {"content-Type":"application/json"}
			 try:
			  	json_data = requests.post(API_address, headers=headers, json=data).json()
			 except requests.exceptions.HTTPError as Exception:
				 print (Exception) 
			 else: population =  json_data['body']['population']
       dispatcher.utter_message(text = population+"is the population of "+country)
       return []               

In [ ]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

In [ ]:
!pip install pyowm

     |████████████████████████████████| 3.1 MB 5.3 MB/s 


In [ ]:
!rasa run actions & rasa shell